# Project - Analyzing Data from CIA World Factbook 2015 with SQL
We'll work with data from the CIA World Factbook 2015, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:
`population` - The population as of 2015.
`population_growth` - The annual population growth rate, as a percentage.
`area` - The total land and water area.

### Load sql and connect to the Database `factbook.db` using following magic 
```
%%capture
%reload_ext sql
%sql sqlite:///factbook.db
```

In [1]:
%%capture
%reload_ext sql
%sql sqlite:///factbook.db
    

### Query the database to get table information for `facts.db` directly:
**Use %%sql magic before query in each code cell to avoid returning an error 

In [2]:
%%sql
SELECT * 
FROM sqlite_master 
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


### Displaying Table Info and first five rows of the Database

In [3]:
%%sql
PRAGMA TABLE_INFO(facts)

 * sqlite:///factbook.db
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,1,None,1
1,code,varchar(255),1,None,0
2,name,varchar(255),1,None,0
3,area,integer,0,None,0
4,area_land,integer,0,None,0
5,area_water,integer,0,None,0
6,population,integer,0,None,0
7,population_growth,float,0,None,0
8,birth_rate,float,0,None,0
9,death_rate,float,0,None,0


In [4]:
%%sql
SELECT *
FROM facts
LIMIT 5;   

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


 - The database contains following columns:

`name` - The name of the country.
`area` - The total land and sea area of the country.
`population` - The country's population.
`population_growth`- The country's population growth as a percentage.
`birth_rate` - The country's birth rate, or the number of births a year per 1,000 people.
`death_rate` - The country's death rate, or the number of death a year per 1,000 people.
`area`- The country's total area (both land and water).
`area_land` - The country's land area in square kilometers.
`area_water` - The country's waterarea in square kilometers.

- Datatypes are dispalyed in the above executed `PRAGMA TABLE_Info(facts'

### Calculating Summary Statistics
#### For population (integer) and population growth (float) statistics, we convert population column to 'float'

In [9]:
%%sql
SELECT MIN(CAST(population AS FLOAT)) AS min_pop, MAX(CAST(population AS FLOAT)) AS max_pop, MIN(population_growth) AS min_pop_grwth, MAX(population_growth) AS max_pop_grwth
FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_grwth,max_pop_grwth
0.0,7256490011.0,0.0,4.02


### Exploring Outliers
- Above statistics show a country with a population of 0 and a country with a population of over 7 billion.
- We will explore these entries further
- Below queries will return countries with minimum and maximum populations

In [10]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MIN(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [11]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The apparently outlying entries above are not erratic as they give populations of Antarctica (uninhabited) and World (total population) respectively.

### Calculating Averages
We will calculate average population and area, but we will convert both columns to `float` type and exclude the row for entire `World` population otherwise it will double the average population.

In [13]:
%%sql
SELECT AVG(CAST(population AS FLOAT)) AS avg_pop, AVG(CAST(area AS FLOAT)) AS avg_area
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


### Finding Densely Populated Countries
We'll identify countries that have:

Above average values for `population`.
Below average values for `area`.

In [17]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(CAST(population AS FLOAT)) FROM facts WHERE name != 'World')
AND AREA < (SELECT AVG(CAST(area AS FLOAT)) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31
